<a href="https://colab.research.google.com/github/softmurata/colab_notebooks/blob/main/vision3d/rgbd23d.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install open3d

In [ ]:
!pip install timm
!pip install ExifRead

In [3]:
import open3d as o3d
import PIL
import exifread
import torch

In [4]:
def create_intrinsic(image_path, default_focal_length=35., sensor_width=36):
    image = PIL.Image.open(image_path)
    img_width, img_height = image.size
    max_side = max(img_width, img_height)
    min_side = min(img_width, img_height)
    with open(image_path, 'rb') as f:
        tags = exifread.process_file(f, details=False)
    if 'EXIF FocalLengthIn35mmFilm' in tags:
        focal_length = tags['EXIF FocalLengthIn35mmFilm'].values[0]
    else:
        print('Could not determine focal length; defaulting to 35.')
        focal_length = default_focal_length
    fx = focal_length * max_side / sensor_width
    # For pinhole cameras fx and fy will be the same.
    fy = fx
    cx = max_side / 2.
    cy = min_side / 2.
    return o3d.camera.PinholeCameraIntrinsic(max_side, min_side, fx, fy, cx, cy)

In [5]:
filename = "room002.jpeg"
intrinsic = create_intrinsic(filename)

In [6]:
intrinsic

PinholeCameraIntrinsic with width = 1000 and height = 667.
Access intrinsics with intrinsic_matrix.

In [7]:
#@title parameters
max_depth = 0.25
model = 'DPT_Large'  # ['DPT_Large', 'DPT_Hybrid', 'MiDaS_small']

In [8]:
#@title midas
midas = torch.hub.load('intel-isl/MiDaS', model)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
midas.to(device)
midas.eval()

midas_transforms = torch.hub.load('intel-isl/MiDaS', 'transforms')

if model == 'DPT_Large' or model == 'DPT_Hybrid':
  transform = midas_transforms.dpt_transform
else:
  transform = midas_transforms.small_transform

/usr/local/lib/python3.8/dist-packages/torch/hub.py:267: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/intel-isl/MiDaS/zipball/master" to /root/.cache/torch/hub/master.zip
Downloading: "https://github.com/isl-org/MiDaS/releases/download/v3/dpt_large_384.pt" to /root/.cache/torch/hub/checkpoints/dpt_large_384.pt


  0%|          | 0.00/1.28G [00:00<?, ?B/s]

Using cache found in /root/.cache/torch/hub/intel-isl_MiDaS_master


In [9]:
import cv2
import numpy as np

In [31]:
filename = "room002.jpeg"
img = cv2.imread(filename)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

input_batch = transform(img).to(device)

with torch.no_grad():
  prediction = midas(input_batch)
  prediction = torch.nn.functional.interpolate(
            prediction.unsqueeze(1),
            size=img.shape[:2],
            mode='bicubic',
            align_corners=False,
        ).squeeze()

depth = prediction.cpu().numpy()

In [33]:
intrinsic = create_intrinsic(filename)

focal = intrinsic.intrinsic_matrix[0, 0]
depth = focal / depth

max_depth = max_depth * depth.max()
depth[depth >= max_depth] = 0

In [34]:
"""
Open3DにRGB-D画像入力
"""
color = o3d.io.read_image(filename)
depth = o3d.geometry.Image(depth)
rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(
    color, depth)
print(rgbd_image)
pcd = o3d.geometry.PointCloud.create_from_rgbd_image(
    rgbd_image,
    o3d.camera.PinholeCameraIntrinsic(
        o3d.camera.PinholeCameraIntrinsicParameters.PrimeSenseDefault))
# Flip it, otherwise the pointcloud will be upside down
pcd.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])

RGBDImage of size 
Color image : 1000x667, with 1 channels.
Depth image : 1000x667, with 1 channels.
Use numpy.asarray to access buffer data.


PointCloud with 645685 points.

In [27]:
# cannot display on colab? what?
import plotly.graph_objects as go

In [28]:
def draw(geometries):
    graph_obj = []

    for gm in geometries:
        geometry_type = gm.get_geometry_type()
        
        if geometry_type == o3d.geometry.Geometry.Type.PointCloud:
            pts = np.asarray(gm.points)
            clr = None  #for colors
            if gm.has_colors():
                clr = np.asarray(gm.colors)
            elif gm.has_normals():
                clr = (0.5, 0.5, 0.5) + np.asarray(gm.normals) * 0.5
            else:
                gm.paint_uniform_color((1.0, 0.0, 0.0))
                clr = np.asarray(gm.colors)

            sc = go.Scatter3d(x=pts[:,0], y=pts[:,1], z=pts[:,2], mode='markers', marker=dict(size=1, color=clr))
            graph_obj.append(sc)

        if geometry_type == o3d.geometry.Geometry.Type.TriangleMesh:
            tri = np.asarray(gm.triangles)
            vert = np.asarray(gm.vertices)
            clr = None
            if gm.has_triangle_normals():
                clr = (0.5, 0.5, 0.5) + np.asarray(gm.triangle_normals) * 0.5
                clr = tuple(map(tuple, clr))
            else:
                clr = (1.0, 0.0, 0.0)
            
            mesh = go.Mesh3d(x=vert[:,0], y=vert[:,1], z=vert[:,2], i=tri[:,0], j=tri[:,1], k=tri[:,2], facecolor=clr, opacity=0.50)
            graph_obj.append(mesh)
        
    fig = go.Figure(
        data=graph_obj,
        layout=dict(
            scene=dict(
                xaxis=dict(visible=False),
                yaxis=dict(visible=False),
                zaxis=dict(visible=False)
            )
        )
    )
    fig.show()

In [29]:
o3d.visualization.draw_geometries = draw # replace function
o3d.visualization.draw_geometries([pcd])

In [35]:
o3d.io.write_point_cloud("room002.ply", pcd, write_ascii=False, compressed=False, print_progress=False)

True

In [ ]:
!pip install plyfile

In [37]:
import numpy as np
import pandas as pd
from plyfile import PlyData

def conver_ply(input_path, output_path):
    plydata = PlyData.read(input_path)  # read file
    data = plydata.elements[0].data  # read data
    data_pd = pd.DataFrame(data)  # convert to DataFrame
    data_np = np.zeros(data_pd.shape, dtype=np.float)  # initialize array to store data
    property_names = data[0].dtype.names  # read names of properties
    for i, name in enumerate(
            property_names):  # read data by property
        data_np[:, i] = data_pd[name]
    data_np.astype(np.float32).tofile(output_path)

In [43]:
conver_ply('/content/roomplan002.ply', '/content/roomplan002.bin')

<ipython-input-37-34a90c8ab136>:9: DeprecationWarning:

`np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations



In [23]:
!wget https://camo.qiitausercontent.com/a8d788734c43a226b7f382effaa284d52cdf894b/68747470733a2f2f71696974612d696d6167652d73746f72652e73332e61702d6e6f727468656173742d312e616d617a6f6e6177732e636f6d2f302f313136343137332f37393863306630392d396639302d323166632d383136632d6138643866636665353862392e6a706567 -O room.jpg

--2023-03-07 07:09:46--  https://camo.qiitausercontent.com/a8d788734c43a226b7f382effaa284d52cdf894b/68747470733a2f2f71696974612d696d6167652d73746f72652e73332e61702d6e6f727468656173742d312e616d617a6f6e6177732e636f6d2f302f313136343137332f37393863306630392d396639302d323166632d383136632d6138643866636665353862392e6a706567
Resolving camo.qiitausercontent.com (camo.qiitausercontent.com)... 13.35.24.83, 13.35.24.3, 13.35.24.59, ...
Connecting to camo.qiitausercontent.com (camo.qiitausercontent.com)|13.35.24.83|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 45632 (45K) [image/jpeg]
Saving to: ‘room.jpg’

room.jpg            100%[===================>]  44.56K  --.-KB/s    in 0.004s  

2023-03-07 07:09:46 (11.2 MB/s) - ‘room.jpg’ saved [45632/45632]



In [42]:
# poisson disk sampling
Mesh = o3d.io.read_triangle_mesh("roomplan002.obj")
# Calculation of normal vector   
Mesh.compute_vertex_normals()

# Mesh to Point Cloud by sample_points_poisson_disk
pcd = Mesh.sample_points_poisson_disk(number_of_points=30000, init_factor=5)
o3d.io.write_point_cloud("roomplan002.ply", pcd)

True